На основе Правил безопасности опасных производственных объектов, на которых используются подъемные сооружения, сделайте нейро-консультанта для ответов на вопросы по представленному документу. Проработайте промт самостоятельно. Проверьте работоспособность созданного консультанта на нескольких самостоятельно сформулированных вопросах. Документ для создания базы знаний: https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc

In [1]:
!pip install faiss-cpu langchain==0.0.271 openai==0.28.1 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [2]:
import os
import getpass
import openai

openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document

In [4]:
def load_google_doc_in_text_format(url: str) -> str:
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [5]:
link_text = 'https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc'
data_from_url= load_google_doc_in_text_format(link_text)

In [6]:
chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
for chunk in splitter.split_text(data_from_url):
    chunks.append(Document(page_content=chunk, metadata={}))

Общее количество чанков:  238


In [7]:
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(chunks, embeddings)

In [8]:
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return " ".join(lines)

def answer_index(system, topic, search_index, temp=1, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос сотрудника на основе представленной информации. Не упоминай документ с информацией для ответа сотруднику в ответе. Документ с информацией для ответа сотруднику: {message_content}\n\nВопрос сотрудника: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer  # возвращает ответ

In [10]:
# Напишите инструкцию для gpt
system="Ты инженер по производственной безопасности, отвечай четко, чтобы сотрудник задавший вопрос понял о чем идет речь"

In [11]:
# вопрос пользователя
topic= "как устанавливаются электрические тали"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================консоли противовеса или от противовеса, расположенного под консолью башенного крана, до площадок, на которых могут находиться люди, должно быть не менее 2 м. 106. Установка электрических талей и монорельсовых тележек с автоматическим или полуавтоматическим управлением, при котором указанное ПС не сопровождается крановщиком или оператором, должна исключать возможность задевания грузом элементов здания, оборудования и штабелей грузов. На пути следования указанных ПС должно быть исключено нахождение людей. Над проезжей частью и над проходами для людей должны быть установлены предохранительные перекрытия (сетка), способные выдержать падающий груз. 107. Установка кранов, передвигающихся по рельсовому пути, в охранной зоне воздушных линий электропередачи должна быть согласована с владельцем линии. Согласование на такую установку для выполнения строительно-монтажных работ должно храни

' Электрические тали устанавливаются в соответствии с требованиями безопасности, изложенными в документе №1. Важно, чтобы установка талей и монорельсовых тележек исключала  возможность задевания грузом элементов здания, оборудования и штабелей грузов. Также необходимо исключить нахождение людей на пути следования этих подъемных средств. Для  безопасности переходов через пути талей должны быть установлены предохранительные перекрытия (сетка), способные выдержать падающий груз. При установке талей на наземном  рельсовом пути, необходимо спланировать основание пути на уровне верха полушпал или установить лоток для кабеля вдоль рельсового пути.'